In [7]:
from langchain_community.document_loaders import JSONLoader
from utils.storer import ConvertPatchNotesToDocuments
import requests
import os

In [8]:
list_patch_notes = os.listdir("patchnotes_modified")
list_patch_notes

['7.20.json',
 '7.20b.json',
 '7.20c.json',
 '7.20d.json',
 '7.20e.json',
 '7.21.json',
 '7.21b.json',
 '7.21c.json',
 '7.21d.json',
 '7.22.json',
 '7.22b.json',
 '7.22c.json',
 '7.22d.json',
 '7.22e.json',
 '7.22f.json',
 '7.22g.json',
 '7.22h.json',
 '7.23.json',
 '7.23a.json',
 '7.23b.json',
 '7.23c.json',
 '7.23d.json',
 '7.23e.json',
 '7.23f.json',
 '7.24.json',
 '7.24b.json',
 '7.25.json',
 '7.25a.json',
 '7.25b.json',
 '7.25c.json',
 '7.26.json',
 '7.26a.json',
 '7.26b.json',
 '7.26c.json',
 '7.27.json',
 '7.27a.json',
 '7.27b.json',
 '7.27c.json',
 '7.27d.json',
 '7.28.json',
 '7.28a.json',
 '7.28b.json',
 '7.28c.json',
 '7.29.json',
 '7.29b.json',
 '7.29c.json',
 '7.29d.json',
 '7.30.json',
 '7.30b.json',
 '7.30c.json',
 '7.30d.json',
 '7.30e.json',
 '7.31.json',
 '7.31b.json',
 '7.31c.json',
 '7.31d.json',
 '7.32.json',
 '7.32b.json',
 '7.32c.json',
 '7.32d.json',
 '7.32e.json',
 '7.33.json',
 '7.33b.json',
 '7.33c.json',
 '7.33d.json',
 '7.33e.json',
 '7.34.json',
 '7.34b.js

In [9]:
docs_all = []
for patch_note in list_patch_notes:
  try:
    print(f"Converting Patch Note {patch_note} to LangChainDoc via JSONLoader")
    converter_pns_to_docs = ConvertPatchNotesToDocuments(patch_note=patch_note)
    docs = converter_pns_to_docs.convert()
    docs_all.append(docs)
  except Exception as e:
    print(f"Error processing patch note {patch_note}: {e}")
    pass

docs_all = [doc for doc_list in docs_all for doc in doc_list]
docs_all

Converting Patch Note 7.20.json to LangChainDoc via JSONLoader
Converting Patch Note 7.20b.json to LangChainDoc via JSONLoader
Converting Patch Note 7.20c.json to LangChainDoc via JSONLoader
Converting Patch Note 7.20d.json to LangChainDoc via JSONLoader
Converting Patch Note 7.20e.json to LangChainDoc via JSONLoader
Converting Patch Note 7.21.json to LangChainDoc via JSONLoader
Converting Patch Note 7.21b.json to LangChainDoc via JSONLoader
Converting Patch Note 7.21c.json to LangChainDoc via JSONLoader
Converting Patch Note 7.21d.json to LangChainDoc via JSONLoader
Converting Patch Note 7.22.json to LangChainDoc via JSONLoader
Converting Patch Note 7.22b.json to LangChainDoc via JSONLoader
Error processing items for patch note 7.22b.json: Cannot iterate over null (null)
Converting Patch Note 7.22c.json to LangChainDoc via JSONLoader
Converting Patch Note 7.22d.json to LangChainDoc via JSONLoader
Error processing items for patch note 7.22d.json: Cannot iterate over null (null)
Convert

[Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 1, 'hero_id': 'Anti-Mage', 'ability_id': None, 'category': 'heroes-abilities', 'patch': '7.20'}, page_content="Ability: None. Hero: Anti-Mage. [{'ability_id': None}, {'ability_notes': ['No updates/changes']}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 2, 'hero_id': 'Axe', 'ability_id': 'axe berserkers call', 'category': 'heroes-abilities', 'patch': '7.20'}, page_content="Ability: axe berserkers call. Hero: Axe. [{'ability_id': 5007, 'ability_notes': [{'indent_level': 1, 'note': 'Armor bonus reduced from 40 to 30'}]}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 3, 'hero_id': 'Bane', 'ability_id': 'bane enfeeble', 'category': 'heroes-abilities', 'patch': '7.20'}, 

In [10]:
docs_all

[Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 1, 'hero_id': 'Anti-Mage', 'ability_id': None, 'category': 'heroes-abilities', 'patch': '7.20'}, page_content="Ability: None. Hero: Anti-Mage. [{'ability_id': None}, {'ability_notes': ['No updates/changes']}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 2, 'hero_id': 'Axe', 'ability_id': 'axe berserkers call', 'category': 'heroes-abilities', 'patch': '7.20'}, page_content="Ability: axe berserkers call. Hero: Axe. [{'ability_id': 5007, 'ability_notes': [{'indent_level': 1, 'note': 'Armor bonus reduced from 40 to 30'}]}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 3, 'hero_id': 'Bane', 'ability_id': 'bane enfeeble', 'category': 'heroes-abilities', 'patch': '7.20'}, 

In [11]:
from typing import Sequence

from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs_all)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [ ]:
### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks about DOTA2 patch notes. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str


def call_model(state: State):
    response = rag_chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context": response["context"],
        "answer": response["answer"],
    }


workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [15]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What are the updates in the 7.37d patch?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
      HumanMessage(content=question),
      AIMessage(content=ai_msg_1["answer"]),
    ]
)

# second_question = "What are common ways of doing it?"
# ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_1["answer"])

I don't know.


In [3]:
# Heroes Abilities

# Extract metadata func for heroes' abilities
def extract_metadata(record: dict, metadata: dict) -> dict:
  metadata["hero_id"] = record.get("hero_id")
  abilities=record.get("abilities", [])
  if abilities:
    metadata["ability_id"] = abilities[0].get("ability_id")
  else:
    metadata["ability_id"] = None
  metadata["category"] = "heroes-abilities"
  return metadata

# Initialize empty list
docs_heroes_abilities = []

for patch_note in list_patch_notes:
  try:
    loader_heroes_abilities = JSONLoader(
      file_path=f"./patchnotes_modified/{patch_note}",
      jq_schema=".heroes[]",
      content_key="abilities",
      text_content=False,
      metadata_func=extract_metadata
    )
    docs_heroes_abilities.append(loader_heroes_abilities.load())
  except Exception as e:
    print(f"Error processing patch note {patch_note}: {e}")
    pass

docs_heroes_abilities = [doc for doc_list in docs_heroes_abilities for doc in doc_list]

# Heroes Abilities Mapper
url_abilities = 'https://api.opendota.com/api/constants/ability_ids'
abilities_response = requests.get(url_abilities)
dict_abilities_map = abilities_response.json()
dict_abilities_map = {key: value.replace('_', ' ') for key, value in dict_abilities_map.items()}

docs_heroes_abilities = replace_id_with_name(docs_heroes_abilities, dict_abilities_map, "ability_id")


docs_heroes_abilities

Error processing patch note 7.23.json: Cannot iterate over null (null)
Error processing patch note 7.26.json: Cannot iterate over null (null)
Error processing patch note 7.26b.json: Cannot iterate over null (null)
Error processing patch note 7.27.json: Cannot iterate over null (null)
Error processing patch note 7.27a.json: Cannot iterate over null (null)
Error processing patch note 7.28.json: Cannot iterate over null (null)


[Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 1, 'hero_id': 1, 'ability_id': None, 'category': 'heroes-abilities'}, page_content="Ability: None. [{'ability_id': None}, {'ability_notes': ['No updates/changes']}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 2, 'hero_id': 2, 'ability_id': 'axe berserkers call', 'category': 'heroes-abilities'}, page_content="Ability: axe berserkers call. [{'ability_id': 5007, 'ability_notes': [{'indent_level': 1, 'note': 'Armor bonus reduced from 40 to 30'}]}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 3, 'hero_id': 3, 'ability_id': 'bane enfeeble', 'category': 'heroes-abilities'}, page_content="Ability: bane enfeeble. [{'ability_id': 5012, 'ability_notes': [{'indent_level': 1, 

In [6]:
# Heroes Talents
def extract_metadata(record: dict, metadata: dict) -> dict:
  metadata["hero_id"] = record.get("hero_id")
  metadata["category"] = "heroes-talents"
  return metadata

# Initialize empty list
docs_heroes_talents = []

for patch_note in list_patch_notes:
  try:
    loader_heroes_talents = JSONLoader(
      file_path=f"patchnotes_modified/{patch_note}",
      jq_schema=".heroes[]",
      content_key="talent_notes",
      text_content=False,
      metadata_func=extract_metadata
    )
    docs_heroes_talents.append(loader_heroes_talents.load())
  except Exception as e:
    print(f"Error processing patch note {patch_note}: {e}")
    pass
  
docs_heroes_talents = [doc for doc_list in docs_heroes_talents for doc in doc_list]
docs_heroes_talents

NameError: name 'list_patch_notes' is not defined

In [17]:
# Heroes Base
def extract_metadata(record: dict, metadata: dict) -> dict:
  metadata["hero_id"] = record.get("hero_id")
  metadata["category"] = "heroes-base"
  return metadata

# Initialize empty list
docs_heroes_base = []

for patch_note in list_patch_notes:
  try:
    loader_heroes_base = JSONLoader(
      file_path=f"patchnotes_modified/{patch_note}",
      jq_schema=".heroes[]",
      content_key="hero_notes",
      text_content=False,
      metadata_func=extract_metadata
    )
    docs_heroes_base.append(loader_heroes_base.load())
  except Exception as e:
    print(f"Error processing patch note {patch_note}: {e}")
    pass

docs_heroes_base = [doc for doc_list in docs_heroes_base for doc in doc_list]
docs_heroes_base

Error processing patch note 7.23.json: Cannot iterate over null (null)
Error processing patch note 7.26.json: Cannot iterate over null (null)
Error processing patch note 7.26b.json: Cannot iterate over null (null)
Error processing patch note 7.27.json: Cannot iterate over null (null)
Error processing patch note 7.27a.json: Cannot iterate over null (null)
Error processing patch note 7.28.json: Cannot iterate over null (null)


[Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 1, 'hero_id': 1, 'category': 'heroes-base'}, page_content="[{'indent_level': 1, 'note': 'Removed Scepter upgrade'}, {'indent_level': 1, 'note': 'Spell Shield replaced with Counterspell'}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 2, 'hero_id': 2, 'category': 'heroes-base'}, page_content="['No updates/changes']"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 3, 'hero_id': 3, 'category': 'heroes-base'}, page_content="['No updates/changes']"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 4, 'hero_id': 4, 'category': 'heroes-base'}, page_content="['No updates/changes']"),
 Docum

In [18]:
url_heroes = 'https://api.opendota.com/api/heroes'
heroes_response = requests.get(url_heroes)
json_heroes_map = heroes_response.json()
df_heroes = pd.DataFrame(json_heroes_map)
dict_heroes_map = df_heroes.set_index('id')['localized_name'].to_dict()
dict_heroes_map

{1: 'Anti-Mage',
 2: 'Axe',
 3: 'Bane',
 4: 'Bloodseeker',
 5: 'Crystal Maiden',
 6: 'Drow Ranger',
 7: 'Earthshaker',
 8: 'Juggernaut',
 9: 'Mirana',
 10: 'Morphling',
 11: 'Shadow Fiend',
 12: 'Phantom Lancer',
 13: 'Puck',
 14: 'Pudge',
 15: 'Razor',
 16: 'Sand King',
 17: 'Storm Spirit',
 18: 'Sven',
 19: 'Tiny',
 20: 'Vengeful Spirit',
 21: 'Windranger',
 22: 'Zeus',
 23: 'Kunkka',
 25: 'Lina',
 26: 'Lion',
 27: 'Shadow Shaman',
 28: 'Slardar',
 29: 'Tidehunter',
 30: 'Witch Doctor',
 31: 'Lich',
 32: 'Riki',
 33: 'Enigma',
 34: 'Tinker',
 35: 'Sniper',
 36: 'Necrophos',
 37: 'Warlock',
 38: 'Beastmaster',
 39: 'Queen of Pain',
 40: 'Venomancer',
 41: 'Faceless Void',
 42: 'Wraith King',
 43: 'Death Prophet',
 44: 'Phantom Assassin',
 45: 'Pugna',
 46: 'Templar Assassin',
 47: 'Viper',
 48: 'Luna',
 49: 'Dragon Knight',
 50: 'Dazzle',
 51: 'Clockwerk',
 52: 'Leshrac',
 53: "Nature's Prophet",
 54: 'Lifestealer',
 55: 'Dark Seer',
 56: 'Clinkz',
 57: 'Omniknight',
 58: 'Enchantress

In [15]:
list_patch_notes = os.listdir("patchnotes_modified")

# Items
def extract_metadata(record: dict, metadata: dict) -> dict:
  metadata["item_id"] = record.get("ability_id")
  metadata["category"] = "items"
  return metadata

# Initialize empty list
docs_items = []

for patch_note in list_patch_notes:
  try:
    loader_items = JSONLoader(
      file_path=f"patchnotes_modified/{patch_note}",
      jq_schema=".items[]",
      content_key="ability_notes",
      is_content_key_jq_parsable=False,
      text_content=False,
      metadata_func=extract_metadata
    )
    docs_items.append(loader_items.load())
  except Exception as e:
    print(f"Error processing patch note {patch_note}: {e}")
    pass

docs_items = [doc for doc_list in docs_items for doc in doc_list]

# Items Mapper
items_url = 'https://api.opendota.com/api/constants/item_ids'
items_response = requests.get(items_url)
dict_item_map = items_response.json()
dict_item_map = {key: value.replace('_', ' ') for key, value in dict_item_map.items()}

# def replace_item_id_with_name(documents, item_id_map):
#   # Loop through each document and replace the item_id with its corresponding name
#   for doc in documents:
#     item_id = doc.metadata.get('item_id')
#     # Convert the item_id to a string to match the dictionary keys
#     item_id_str = str(item_id)
#     # Replace item_id with corresponding value from the dictionary, if available
#     if item_id_str in item_id_map:
#       doc.metadata['item_id'] = item_id_map[item_id_str]  # Replace with the item name4
#     doc.page_content = f"Item: {doc.metadata['item_id']}. {doc.page_content}"  
#   return documents

# docs_items = replace_item_id_with_name(docs_items, dict_item_map)
docs_items

Error processing patch note 7.22b.json: Cannot iterate over null (null)
Error processing patch note 7.22d.json: Cannot iterate over null (null)
Error processing patch note 7.23.json: Cannot iterate over null (null)
Error processing patch note 7.23e.json: Cannot iterate over null (null)
Error processing patch note 7.23f.json: Cannot iterate over null (null)
Error processing patch note 7.25a.json: Cannot iterate over null (null)
Error processing patch note 7.26.json: Cannot iterate over null (null)
Error processing patch note 7.26c.json: Cannot iterate over null (null)
Error processing patch note 7.28.json: Cannot iterate over null (null)
Error processing patch note 7.28a.json: Cannot iterate over null (null)
Error processing patch note 7.28c.json: Cannot iterate over null (null)
Error processing patch note 7.29b.json: Cannot iterate over null (null)
Error processing patch note 7.33e.json: Cannot iterate over null (null)


[Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 1, 'item_id': 50, 'category': 'items'}, page_content="[{'indent_level': 1, 'note': 'Reworked item ability and recipe'}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 2, 'item_id': 63, 'category': 'items'}, page_content="[{'indent_level': 1, 'note': 'Reworked item recipe and bonuses'}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 3, 'item_id': 214, 'category': 'items'}, page_content="[{'indent_level': 1, 'note': 'No longer disabled by attacking creeps'}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 4, 'item_id': 162, 'category': 'items'}, page_content="[{'indent_level': 1, 

In [12]:
def add_patch_metadata(documents):
  for doc in documents:
    # Extract the patch version from the source (assuming file path is in 'source')
    source = doc.metadata.get("source", "")
    if source:
      patch_version = source.split("\\")[-1].replace(".json", "")
      doc.metadata["patch"] = patch_version
  return documents

docs_items = add_patch_metadata(docs_items)

In [13]:
docs_items

[Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 1, 'item_id': 'phase boots', 'category': 'items', 'patch': '7.20'}, page_content="Item: phase boots. [{'indent_level': 1, 'note': 'Reworked item ability and recipe'}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 2, 'item_id': 'power treads', 'category': 'items', 'patch': '7.20'}, page_content="Item: power treads. [{'indent_level': 1, 'note': 'Reworked item recipe and bonuses'}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-notes-assistant\\patchnotes_modified\\7.20.json', 'seq_num': 3, 'item_id': 'tranquil boots', 'category': 'items', 'patch': '7.20'}, page_content="Item: tranquil boots. [{'indent_level': 1, 'note': 'No longer disabled by attacking creeps'}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-d

In [35]:
docs_all = docs_heroes_base + docs_heroes_abilities + docs_heroes_talents + docs_items

In [7]:
def ensure_page_content_is_string(documents):
  # Loop through each document and make sure page_content is a string
  for doc in documents:
      if not isinstance(doc.page_content, str):
          doc.page_content = str(doc.page_content)  # Convert to string if it's not already a string
  return documents

# Ensure all page_content fields are strings
docs_all = ensure_page_content_is_string(docs_all)

NameError: name 'docs_all' is not defined

In [56]:
docs_all

[Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 1, 'hero_id': 1, 'category': 'heroes-base'}, page_content="['No updates/changes']"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 2, 'hero_id': 2, 'category': 'heroes-base'}, page_content="[{'indent_level': 1, 'note': 'Base Armor increased by 1'}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 3, 'hero_id': 3, 'category': 'heroes-base'}, page_content="['No updates/changes']"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 4, 'hero_id': 4, 'category': 'heroes-base'}, page_content="['No updates/changes']"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-

In [18]:
from typing import Sequence

import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [19]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs_items)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [20]:
### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks about DOTA2 patch notes. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str


def call_model(state: State):
    response = rag_chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context": response["context"],
        "answer": response["answer"],
    }


workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [21]:
docs_items

[Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 1, 'item_id': 'ring of tarrasque', 'category': 'items', 'patch': '7.35'}, page_content="Item: ring of tarrasque. [{'indent_level': 2, 'note': 'Now is a basic Secret Shop item. Cost: 1800'}, {'indent_level': 2, 'note': 'Provides +12 Health Regen'}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 2, 'item_id': 'tiara of selemene', 'category': 'items', 'patch': '7.35'}, page_content="Item: tiara of selemene. [{'indent_level': 2, 'note': 'New basic Secret Shop item. Cost: 1800'}, {'indent_level': 2, 'note': 'Provides +6 Mana Regen'}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 3, 'item_id': 'angels demise', 'category': 'items', 'patch': '7.35'}, page_content='Item: angels demi

In [22]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is the tiara of selemene in 7.34 patch?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
      HumanMessage(content=question),
      AIMessage(content=ai_msg_1["answer"]),
    ]
)

# second_question = "What are common ways of doing it?"
# ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_1["answer"])

In the 7.34 patch, the Tiara of Selemene is a new basic Secret Shop item that costs 1800 gold and provides +6 Mana Regen.


In [23]:

### Construct retriever ###
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()


### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str


def call_model(state: State):
    response = rag_chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context": response["context"],
        "answer": response["answer"],
    }


workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [6]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Common ways of task decomposition include using simple prompting techniques, such as asking for "Steps for XYZ" or "What are the subgoals for achieving XYZ?" Additionally, task-specific instructions can be employed, like "Write a story outline" for writing tasks, or human inputs can guide the decomposition process.


In [ ]:
#Sample